In [1]:
import pandas as pd
import requests
import json 
import re


In [2]:
def Find_url(string):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)
    return [x[0] for x in url]

In [3]:
url = "https://www.aponet.de/apotheke/apothekensuche?tx_aponetpharmacy_search%5Baction%5D=result&tx_aponetpharmacy_search%5Bcontroller%5D=Search&tx_aponetpharmacy_search%5Bsearch%5D%5Bplzort%5D=22113&tx_aponetpharmacy_search%5Bsearch%5D%5Bstreet%5D=&tx_aponetpharmacy_search%5Bsearch%5D%5Bradius%5D=50&tx_aponetpharmacy_search%5Bsearch%5D%5Blat%5D=&tx_aponetpharmacy_search%5Bsearch%5D%5Blng%5D=&type=1981"
page = requests.get(url)
data = json.loads(page.text)


In [4]:
if len(Find_url(data['features'][0]['properties']['popup'])) != 0:
    apo_url = Find_url(data['features'][0]['properties']['popup'])[0].split(';')[0][:-5]
else:
    apo_url = None


In [5]:
ls_dc = []
for i in range(len(data['features'])):
    apothieka_name = data['features'][i]['properties']['name']
    apothieka_address = data['features'][i]['properties']['strasse']
    apothieka_plz = data['features'][i]['properties']['plz']
    apothieka_tel = data['features'][i]['properties']['telefon']
    apothieka_email = data['features'][i]['properties']['email']
    
    if len(Find_url(data['features'][i]['properties']['popup'])) != 0:
        apothieka_url = Find_url(data['features'][i]['properties']['popup'])[0].split(';')[0][:-5]
    else:
        apothieka_url = None
        
    ls_dc.append({'apothieka_name': apothieka_name,
                          'apothieka_address':apothieka_address,
                          'apothieka_plz': apothieka_plz,
                          'apothieka_tel': apothieka_tel,
                          'apothieka_email': apothieka_email,
                          'apothieka_url': apothieka_url,
                          })
  


In [6]:
df = pd.DataFrame(ls_dc)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   apothieka_name     738 non-null    object
 1   apothieka_address  738 non-null    object
 2   apothieka_plz      738 non-null    object
 3   apothieka_tel      738 non-null    object
 4   apothieka_email    738 non-null    object
 5   apothieka_url      245 non-null    object
dtypes: object(6)
memory usage: 34.7+ KB


In [8]:
df.to_csv('apothieka.csv', index=False)